In [2]:
#Importing all the necessary packages required for pre-processing:
import numpy as np #Used for array operations
import nltk #Used as a basic package for nlp operations
from nltk.corpus import stopwords #Helps in stop words removal
from nltk.stem import WordNetLemmatizer #Helps in lemmatization process
from nltk.stem import PorterStemmer #Helps in stemming process
#Helps in tokenization of words:
from nltk.tokenize import word_tokenize 
from nltk.tokenize import sent_tokenize
import math #Used for mathematical equations solving
import pickle #Used to dump files to memory so, that we don't need to train model again anad again for a long time.
import os #Used to iterate files in the local system
import glob #Used for identifying paths in local system
import re #Used for removal of non-ascii characters
import sys #Used to manipulate different parts of the Python runtime environment
from pathlib import Path #Makes it very easy and efficient to deal with file paths
from collections import Counter #Used for carrying out frequency count

In [3]:
stop=set(stopwords.words('english')) #Storing all stop words in a set data structure
ps=PorterStemmer() #Creating object of PorterStemmer

In [4]:
#Loading pickle file and saving it as a variable:
temp=open('Saved/posting_list.pkl',"rb")
posting_lists=pickle.load(temp)

#Loading pickle file and saving it as a variable:
temp=open('Saved/file_idx.pkl','rb')
file_index=pickle.load(temp)

In [5]:
unique_words=set(posting_lists.keys()) #The keys of posting_list are stored in the set to get unique words existing

In [6]:
#Defining some functions that we'll use in future:
def remove_non_ascii_characters(data):
    pattern=re.compile('[^a-zA-Z0-9\s]')
    out=re.sub(pattern,'',data)
    return out

#stemming/Lemmatize
def doStemming(words):
    from nltk.stem import PorterStemmer
    ps=PorterStemmer()
    
    stem_list=[ps.stem(w) for w in words]
    return stem_list

def removeStopWords(words):
    from nltk.corpus import stopwords
    stop_words=set(stopwords.words('english'))
    word_list=[]
    for w in words:
        if w=="and" or w=="or" or (w not in stop_words):
            word_list.append(w)
    return word_list

def convertToLower(text):
    return str(np.char.lower(text))

def numToWords(words):
    for i,w in enumerate(words):
        if w.isdigit() and len(w)<4:
            words[i]=num2words(int(w))
    return words

In [7]:
#Taking input query from user and storing it as a variable. 
query=input("Enter your query: ")

data = remove_non_ascii_characters(query) #Removing special characters
data = re.sub(re.compile('\d'),'',data)
words = word_tokenize(data) #Tokenization 
words = [word.lower() for word in words] #Lower casing
words = [ps.stem(word) for word in words] #Stemming
words = [word for word in words if word not in stop] #Stop words removal

print(words) #print the words

Enter your query: shift register flip flop
['shift', 'regist', 'flip', 'flop']


In [8]:
if len(words)>1:
    query_words=[words[0]]
else:
    query_words=words

for i in range(1,len(words)):
    if words[i] not in ["and","or"]:
        if query_words[-1] not in ["and","or"]:
            query_words.append("and")
            query_words.append(words[i])
        else:
            query_words.append(words[i])
    elif query_words[-1] not in ["and","or"]:
        query_words.append(words[i])

print(query_words)

['shift', 'and', 'regist', 'and', 'flip', 'and', 'flop']


In [9]:
operators=[]
main_words=[]

for w in query_words:
    if w.lower() in ["and","or"]:
        operators.append(w.lower())
    else:
        main_words.append(w)
    
print(operators)
print(main_words)

['and', 'and', 'and']
['shift', 'regist', 'flip', 'flop']


In [10]:
n=len(file_index)
word_vector=[]
word_vector_matrix=[]

for w in main_words:
    word_vector=[0]*n
    if w in unique_words:
        for x in posting_lists[w].keys():
            word_vector[x]=1
    word_vector_matrix.append(word_vector)

In [11]:
for w in operators:
    vector1=word_vector_matrix[0]
    vector2=word_vector_matrix[1]
    
    if w=="and":
        result=[b1&b2 for b1,b2 in zip(vector1,vector2)]
        
        word_vector_matrix.pop(0)
        word_vector_matrix.pop(0)
        
        word_vector_matrix.insert(0,result)
    else:
        result=[b1|b2 for b1,b2 in zip(vector1,vector2)]
        
        word_vector_matrix.pop(0)
        word_vector_matrix.pop(0)
        
        word_vector_matrix.insert(0,result)

In [12]:
final_word_vector=word_vector_matrix[0]
cnt=0
files=[]
for i in final_word_vector:
    if i==1:
        files.append(file_index[cnt])
    cnt+=1

In [13]:
files

['C00105.txt', 'C00723.txt']